#### Dermatology Model (WIP)

Descrição: 

In [2]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


In [ ]:
# --- 1. Carregar e Inspecionar o Dataset ---
print("--- 1. Carregando e Inspecionando o Dataset Dermatology ---")
car_evaluation = fetch_ucirepo(id=33)
X_original = car_evaluation.data.features
y_original = car_evaluation.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")
print("-" * 50)

--- 1. Carregando e Inspecionando o Dataset Car Evaluation ---


Dimensão inicial de X: (1728, 6)
Colunas de X e seus tipos de dados:
buying      object
maint       object
doors       object
persons     object
lug_boot    object
safety      object
dtype: object

Primeiras 5 linhas de X:
  buying  maint doors persons lug_boot safety
0  vhigh  vhigh     2       2    small    low
1  vhigh  vhigh     2       2    small    med
2  vhigh  vhigh     2       2    small   high
3  vhigh  vhigh     2       2      med    low
4  vhigh  vhigh     2       2      med    med

Classes de y:
class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64
--------------------------------------------------


In [4]:
# --- 2. Pré-processamento: One-Hot Encoding para Atributos Categóricos ---
print("--- 2. Pré-processamento: One-Hot Encoding ---")

# O Car Evaluation tem todas as colunas como 'object' (categóricas).
# O OneHotEncoder lidará com todas elas.
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False retorna array denso
# sparse_output=False é o mesmo que chamar .toarray() depois.
# Não precisamos do .astype(int).tolist() ainda, faremos a conversão de tipo para np.uint8 depois do split.
X_encoded_dense = encoder.fit_transform(X_original)

print(f"Dimensão de X após One-Hot Encoding: {X_encoded_dense.shape}")
print(f"Tipo dos elementos após One-Hot Encoding: {X_encoded_dense.dtype}")
print(f"Primeiras 2 linhas de X_encoded_dense:\n{X_encoded_dense[:2]}")
print("-" * 50)

--- 2. Pré-processamento: One-Hot Encoding ---
Dimensão de X após One-Hot Encoding: (1728, 21)
Tipo dos elementos após One-Hot Encoding: float64
Primeiras 2 linhas de X_encoded_dense:
[[0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]]
--------------------------------------------------


In [5]:
# --- 3. Divisão dos Dados em Treino e Teste ---
print("--- 3. Dividindo os Dados em Treino e Teste ---")

# Para o WiSARD, os dados de entrada X devem ser do tipo np.uint8.
# E os rótulos y devem ser strings.
# Fazemos a conversão de tipo após o split para manter a consistência do tipo de dados.
X_encoded_uint8 = X_encoded_dense.astype(np.uint8)
y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")
print("-" * 50)

--- 3. Dividindo os Dados em Treino e Teste ---
Tamanho do conjunto de treino (X_train): 518 amostras, 21 bits
Tamanho do conjunto de teste (X_test): 1210 amostras, 21 bits
Tipo dos elementos de X_train: uint8
--------------------------------------------------


In [7]:
# --- 4. Configuração e Treinamento do Modelo WiSARD ---
print("--- 4. Configurando e Treinando o Modelo WiSARD ---")

# addressSize (tuple_size) para o WiSARD.
# O número de bits de entrada de X_encoded_dense para Car Evaluation é 21.
# Um addressSize de 21 ainda é muito grande.
# Escolha um addressSize menor, como 2, 3 ou 4.
# Por exemplo, se X_encoded_dense tem 21 bits, e addressSize=3, teremos 21/3 = 7 RAMs por classe.
addressSize = 3 # Ajuste este valor. 2 ou 3 são bons pontos de partida.

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
print("Treinando o WiSARD...")
wsd.train(X_train, y_train)
print("Treinamento concluído.")
print("-" * 50)


# --- 5. Classificação e Avaliação do Modelo WiSARD ---
print("--- 5. Classificação e Avaliação do Modelo WiSARD ---")

print("Classificando os dados de teste...")
predictions_str = wsd.classify(X_test)
print("Classificação concluída.")

# Convertendo as previsões de string de volta para um array NumPy para fácil comparação
predictions_np = np.array(predictions_str)
y_test_np = np.array(y_test) # Converte o y_test também para numpy array para comparação elemento a elemento

accuracy_wisard = np.mean(predictions_np == y_test_np)
print(f"Acurácia do WiSARD: {accuracy_wisard:.4f}")
print("-" * 50)


# --- 6. Comparação com o Multi-Layer Perceptron (MLP) ---
print("--- 6. Comparação com o Multi-Layer Perceptron (MLP) ---")
# Por favor, substitua '0.92 (Exemplo MLP)' pelo resultado real do seu MLP para este dataset.
mlp_accuracy_car_evaluation = "0.92 (Exemplo MLP)" # Substitua com seu valor real

print(f"Acurácia do MLP (comparação): {mlp_accuracy_car_evaluation}")
print("\nObservação: Para uma comparação justa, certifique-se de que o MLP foi treinado e testado")
print("nas mesmas condições (divisão de dados, pré-processamento, etc.).")
print("-" * 50)

--- 4. Configurando e Treinando o Modelo WiSARD ---
WiSARD inicializado com addressSize=3
Treinando o WiSARD...
training 106 of 5Treinamento concluído.
--------------------------------------------------
--- 5. Classificação e Avaliação do Modelo WiSARD ---
Classificando os dados de teste...
Classificação concluída.
training 156 of 5Acurácia do WiSARD: 0.7306
--------------------------------------------------
--- 6. Comparação com o Multi-Layer Perceptron (MLP) ---
Acurácia do MLP (comparação): 0.92 (Exemplo MLP)

Observação: Para uma comparação justa, certifique-se de que o MLP foi treinado e testado
nas mesmas condições (divisão de dados, pré-processamento, etc.).
--------------------------------------------------
training 518 of 51818
classifying 494 of 1210

classifying 1210 of 1210
